# Mount Drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

# Set Environment

In [1]:
%cd {'drive/MyDrive/DJIA'}

/content/drive/MyDrive/DJIA


# Load Libraries

In [2]:
%load_ext autoreload
%autoreload 2

# load the data
import pandas as pd

# load the model
import joblib

# Load data

In [3]:
!pip install --upgrade -q tables
data = pd.read_hdf('data/ts_cl.h5', 'all_features')
# avoid sklearn future warning
data.columns = data.columns.astype(str)
data.drop('fwd_returns', axis=1, inplace=True)

# Load Model

You can load your desired model from the djia_models folder. The procedure is the same.

In [4]:
ensemble = joblib.load('djia_models/StackingEnsemble.joblib')

# Prepare data for model

In [5]:
window=60
sequence = list(range(1, window+1))
sequence = [str(x) for x in sequence]

oob_data = data.loc['2016-04':] 
X_oob_lstm = [
           oob_data.loc[:, sequence].values.reshape(-1, window, 1),  # get first window returns
           oob_data.filter(like='topic'),  # get main news topics
           oob_data.filter(like='ind')     # indicators from ts
]
y_oob_lstm = oob_data.Label

# Predict

In [6]:
yhat = ensemble.predict(X_oob_lstm)  # get all predictions

In [8]:
pred_vs_test = pd.DataFrame({'Test_labels' : y_oob_lstm, 'Pred_labels':yhat}) 
pred_vs_test.sample(10)  # get a sample of 10 predicitons

,Test_labels,Pred_labels
2016-05-10,0,0
2016-06-22,1,1
2016-06-08,1,1
2016-05-18,1,1
2016-04-12,0,0
2016-06-24,1,0
2016-04-20,1,1
2016-05-02,1,1
2016-06-13,1,1
2016-05-25,0,0
